In [ ]:
!pip3 install -e /Users/gva/Documents/PhD/workspace/target/misc/sgd/

In [ ]:
from sgd import *
import time
from numpy import genfromtxt
# from sklearn.preprocessing import normalize

In [ ]:
#data = sm.datasets.scotland.load()
#data.exog = sm.add_constant(data.exog, prepend=False)
#D = data_set(data.exog, data.endog)

In [ ]:
data = genfromtxt('glm_test.csv', delimiter=',', encoding='UTF-8', skip_header=1)

In [ ]:
D = data_set(data[:,1:], data[:,0])

In [ ]:
#data = sm.datasets.scotland.load(as_pandas=True)
#data.exog = sm.add_constant(data.exog, prepend=False)
#data.exog["RESPONSE"] = data.endog
#data.exog.to_csv(os.path.join("~/Documents/PhD/workspace/target/misc/glm_test.csv"), index=False, encoding='UTF-8')

In [ ]:
m = glm(0.0, 0.0, "gaussian", "identity")

In [ ]:
glm_bnch = sm.GLM(D._Y, D._X, family = sm.families.Gaussian())
true_est = glm_bnch.fit()

In [ ]:
print(true_est.summary())

In [ ]:
timer = time
n, p = D._X.shape

details = {"method": "explicit", "nparams": p, "reltol": 1e-10, "npasses": 10,\
               "size": 10, "pass": True, "check": True, "truth": np.ones(p) }

# tester = ExplicitSGD(n, timer, **details)
tester = ImplicitSGD(n, timer, **details)

In [ ]:
n_samples = n
n_features = p
n_passes = tester.get_value_of("n_passes")

In [ ]:
good_gradient = True
good_validity = True
averaging = True

In [ ]:
theta_old = np.ones(p) / 10.0
theta_old_ave = theta_old
max_iters = n_samples * n_passes
do_more_iterations = True
converged = False

t = 1

theta_new = np.ones(p)
theta_new_ave = np.ones(p)
while do_more_iterations:
    theta_new = tester.update(t, theta_old, D, m, good_gradient)
    if averaging:
        if t != 1: theta_new_ave = (1.0 - 1.0/float(t)) * theta_old_ave + (1.0/float(t)) * theta_new
        else: theta_new_ave = theta_new
        tester.sync_members(theta_new_ave)
        converged = tester.convergence(theta_new_ave, theta_old_ave)
        theta_old_ave = theta_new_ave
    else:
        tester.sync_members(theta_new)
        converged = tester.convergence(theta_new, theta_old)
    if converged: do_more_iterations = False
    theta_old = theta_new
    if t == max_iters: do_more_iterations = False
    t += 1

In [ ]:
theta_new

In [ ]:
converged